# 自定义联邦学习算法

AlphaMed 平台内置了一些常用的算法调度器实现。然而现实世界的需求千变万化，内置实现并不总能满足现实任务的需要。为此 AlphaMed 平台提供了对自定义联邦算法的支持。参考以下介绍的流程，工程师可以根据自己的业务需要定制各种联邦学习算法。

**以下说明仅限于算法计算流程，不涉及计算开始之前的数据验证操作。**

在 AlphaMed 平台自定义联邦学习算法并实现算法对应的调度器主要有以下几个步骤：
1. 定义联邦学习算法流程；
2. 根据算法流程，定义调度流程和各个参与角色的流程；
3. 根据调度流程，定义用于互相协调的消息；
4. 依据第 3 步定义的合约消息，实现对应的合约消息体、消息工厂、消息发生工具；（关于自定义合约消息的内容看[这里](6.%20%E5%90%88%E7%BA%A6%E6%B6%88%E6%81%AF%E6%9C%BA%E5%88%B6%E7%AE%80%E4%BB%8B.ipynb)）
5. 设计流程中需要用户实现的接口；
6. 实现流程调度代码逻辑。

下面按照上述步骤逐个演示。

## 定义联邦学习算法流程

这里定义一个极简版的 FedAvg 流程，仅提供按照设置的轮次训练和聚合的功能，重启恢复、异常处理、安全隐私等其它功能均不考虑。但已经足够演示如何设计一个算法并将其在 AlphaMed 平台上实现。定义算法流程时先不考虑实际运行时的外部约束，比如节点集合同步、AlphaMed 平台要求等，以避免处理逻辑过于复杂，影响了对核心流程的理解。

算法流程如下：
```
聚合方初始化全局模型、参与方初始化本地模型；  
for 设置的训练轮次：  
    聚合方向所有参与方发送全局模型；  
    参与方接收全局模型，使用其更新本地模型；  
    参与方在本地做一个 epoch 的训练，更新本地模型；  
    参与方将本地模型发送给聚合方；  
    聚合方平均所有参与方的本地模型，得到新的全局模型；  
    聚合方使用测试数据做一次测试，得到全局模型的评估指标数据；
```

## 根据算法流程，定义调度流程和各个参与角色的流程

定义调度流程时，就需要考虑算法在 AlphaMed 平台上实际运行时的情况了。此时会涉及节点集合同步、AlphaMed 平台要求等外部约束，需要一并考虑。

加入外部约束后的流程为：
1. 聚合方和所有参与方上线，初始化本地资源；（各参与方上线顺序是随机的，可能有各种情况。）
2. 此时参与方还不知道聚合方是谁，所以广播发送集合请求；
3. 聚合方收到集合请求后，记录参与者，并向请求方回复集合响应；
4. 参与方收到集合响应后，记录聚合方，等待聚合方通知开始训练；
5. 聚合方统计参与者数量，如果集合完成发送开始训练的通知；
6. 开始训练后，聚合方、参与方完成初始化工作；
7. 聚合方计数训练轮次，发送全局模型；
8. 参与方接收全局模型，使用其更新本地模型；
9. 参与方在本地做一个 epoch 的训练，更新本地模型；
10. 参与方将本地模型发送给聚合方；
11. 聚合方收集本地模型，集齐全部本地模型后，聚合得到新的全局模型；
12. 聚合方检查训练轮次，判断训练是否完成；如果没有完成则跳到第 7 步，如果完成继续向下；
13. 聚合方使用测试数据做一次测试，得到全局模型的评估指标数据；
14. 聚合方上传模型文件和评估指标数据；
15. 聚合方通知任务管理器训练结束，完成训练退出；
16. 参与方收到训练结束的通知，完成训练退出。

在此基础上可以拆解出聚合方、参与方各自的流程。

聚合方流程为：
1. 上线，初始化本地资源；
2. 监听集合请求；
3. 收到集合请求后，记录参与者，并向请求方回复集合响应；
4. 统计参与者数量，如果集合完成发送开始训练的通知；
5. 完成训练初始化工作；
6. 计数训练轮次，发送全局模型；
7. 监听本地模型传输请求，收集本地模型，集齐全部本地模型后，聚合得到新的全局模型；
8. 检查训练轮次，判断训练是否完成；如果没有完成则跳到第 6 步，如果完成继续向下；
9. 使用测试数据做一次测试，得到全局模型的评估指标数据；
10. 上传模型文件和评估指标数据；
11. 通知任务管理器训练结束，完成训练退出。

参与方流程为：
1. 上线，初始化本地资源；
2. 此时还不知道聚合方是谁，所以广播发送集合请求；
3. 监听集合响应，收到集合响应后，记录聚合方；
4. 完成训练初始化工作；
5. 监听开始训练消息或训练结束消息；如果收到开始训练消息，跳到第 6 步；如果收到训练结束消息，完成训练退出；
6. 监听传输全局模型消息；
7. 接收全局模型，使用其更新本地模型；
8. 在本地做一个 epoch 的训练，更新本地模型；
9. 将本地模型发送给聚合方；
16. 跳到第 5 步。

## 根据调度流程，定义用于互相协调的消息

再贴一遍调度流程。
1. 聚合方和所有参与方上线，初始化本地资源；（上线顺序随意，可能有各种情况）
2. 此时参与方还不知道聚合方是谁，所以广播发送集合请求；
3. 聚合方收到集合请求后，记录参与者，并向请求方回复集合响应；
4. 参与方收到集合响应后，记录聚合方，等待聚合方通知开始训练；
5. 聚合方统计参与者数量，如果集合完成发送开始训练的通知；
6. 开始训练后，聚合方、参与方完成初始化工作；
7. 聚合方计数训练轮次，发送全局模型；
8. 参与方接收全局模型，使用其更新本地模型；
9. 参与方在本地做一个 epoch 的训练，更新本地模型；
10. 参与方将本地模型发送给聚合方；
11. 聚合方收集本地模型，集齐全部本地模型后，聚合得到新的全局模型；
12. 聚合方检查训练轮次，判断训练是否完成；如果没有完成则跳到第 7 步，如果完成继续向下；
13. 聚合方使用测试数据做一次测试，得到全局模型的评估指标数据；
14. 聚合方上传模型文件和评估指标数据；
15. 聚合方通知任务管理器训练结束，完成训练退出；
16. 参与方收到训练结束的通知，完成训练退出。

根据调度流程，整理出需要以下消息来协调各方动作：集合请求消息、集合响应消息、开始训练消息、训练结束消息。

## 实现合约消息体、消息工厂、消息发生工具

参考[合约消息机制简介](6.%20%E5%90%88%E7%BA%A6%E6%B6%88%E6%81%AF%E6%9C%BA%E5%88%B6%E7%AE%80%E4%BB%8B.ipynb)，代码入下：

In [ ]:
from dataclasses import dataclass
from alphafed.contractor import ContractEvent


@dataclass
class CheckInEvent(ContractEvent):
    """集合请求消息。"""

    TYPE = 'check_in'

    peer_id: str

    @classmethod
    def contract_to_event(cls, contract: dict) -> 'CheckInEvent':
        event_type = contract.get('type')
        peer_id = contract.get('peer_id')
        assert event_type == cls.TYPE, f'合约类型错误: {event_type}'
        assert peer_id and isinstance(peer_id, str), f'invalid peer_id: {peer_id}'
        return CheckInEvent(peer_id=peer_id)


@dataclass
class CheckInResponseEvent(ContractEvent):
    """集合响应消息。"""

    TYPE = 'check_in_resp'

    aggr_id: str

    @classmethod
    def contract_to_event(cls, contract: dict) -> 'CheckInResponseEvent':
        event_type = contract.get('type')
        aggr_id = contract.get('aggr_id')
        assert event_type == cls.TYPE, f'合约类型错误: {event_type}'
        assert aggr_id and isinstance(aggr_id, str), f'invalid aggr_id: {aggr_id}'
        return CheckInResponseEvent(aggr_id=aggr_id)


@dataclass
class StartEvent(ContractEvent):
    """开始训练消息。"""

    TYPE = 'start'

    @classmethod
    def contract_to_event(cls, contract: dict) -> 'StartEvent':
        event_type = contract.get('type')
        assert event_type == cls.TYPE, f'合约类型错误: {event_type}'
        return StartEvent()


@dataclass
class CloseEvent(ContractEvent):
    """训练结束消息。"""

    TYPE = 'close'

    @classmethod
    def contract_to_event(cls, contract: dict) -> 'CloseEvent':
        event_type = contract.get('type')
        assert event_type == cls.TYPE, f'合约类型错误: {event_type}'
        return CloseEvent()

In [ ]:
from alphafed.contractor import TaskMessageEventFactory


class SimpleFedAvgEventFactory(TaskMessageEventFactory):

    _CLASS_MAP = {
        CheckInEvent.TYPE: CheckInEvent,
        CheckInResponseEvent.TYPE: CheckInResponseEvent,
        StartEvent.TYPE: StartEvent,
        CloseEvent.TYPE: CloseEvent,
        **TaskMessageEventFactory._CLASS_MAP
    }

In [ ]:
from alphafed.contractor import TaskMessageContractor

class SimpleFedAvgContractor(TaskMessageContractor):

    def __init__(self, task_id: str):
        super().__init__(task_id=task_id)
        self._event_factory = SimpleFedAvgEventFactory

    def check_in(self, peer_id: str):
        """发送集合请求消息。"""
        event = CheckInEvent(peer_id=peer_id)
        self._new_contract(targets=self.EVERYONE, event=event)

    def response_check_in(self, aggr_id: str, peer_id: str):
        """发送集合响应消息。"""
        event = CheckInResponseEvent(aggr_id=aggr_id)
        self._new_contract(targets=[peer_id], event=event)

    def start(self):
        """发送开始训练消息。"""
        event = StartEvent()
        self._new_contract(targets=self.EVERYONE, event=event)

    def close(self):
        """发送训练结束消息。"""
        event = CloseEvent()
        self._new_contract(targets=self.EVERYONE, event=event)

将合约消息代码整理好，集中放在一个 [contractor.py](res/simple_fed_avg/contractor.py) 文件中，方便使用。

## 设计流程中需要用户实现的接口

再贴一遍调度流程。
1. 聚合方和所有参与方上线，初始化本地资源；（上线顺序随意，可能有各种情况）
2. 此时参与方还不知道聚合方是谁，所以广播发送集合请求；
3. 聚合方收到集合请求后，记录参与者，并向请求方回复集合响应；
4. 参与方收到集合响应后，记录聚合方，等待聚合方通知开始训练；
5. 聚合方统计参与者数量，如果集合完成发送开始训练的通知；
6. 开始训练后，聚合方、参与方；
7. 聚合方计数训练轮次，发送全局模型；
8. 参与方接收全局模型，使用其更新本地模型；
9. 参与方在本地做一个 epoch 的训练，更新本地模型；
10. 参与方将本地模型发送给聚合方；
11. 聚合方收集本地模型，集齐全部本地模型后，聚合得到新的全局模型；
12. 聚合方检查训练轮次，判断训练是否完成；如果没有完成则跳到第 7 步，如果完成继续向下；
13. 聚合方使用测试数据做一次测试，得到全局模型的评估指标数据；
14. 聚合方上传模型文件和评估指标数据；
15. 聚合方通知任务管理器训练结束，完成训练退出；
16. 参与方收到训练结束的通知，完成训练退出。

仔细梳理上述流程，可以整理出以下这些操作，是算法调度流程本身处理不了的，需要使用者提供对应的逻辑。那就将这些方法封装为接口，由使用者提供实现逻辑，而调度器只需要在对应的流程节点上调用即可。
- 完成集合前初始化本地资源；
- 训练开始前的初始化工作；
- 获取训练使用的模型对象；
- 完成一个 epoch 训练的逻辑；
- 测试的逻辑。

因此逐个定义接口。由于现在还没有涉及实现细节，可以先不考虑接口的具体参数，待将来实现时补充完善。注意这里只是简单的示例，因此没有对接口设计做优化。现实中设计真正可用的算法调度器时，设计者可以根据自身理解优化接口设计。

In [ ]:
from abc import ABCMeta, abstractmethod

@abstractmethod
def before_check_in(self):
    """完成集合前初始化本地资源。"""

@abstractmethod
def before_training(self):
    """训练开始前的初始化工作。"""

@property
@abstractmethod
def model(self):
    """获取训练使用的模型对象。"""

@abstractmethod
def train_an_epoch(self):
    """完成一个 epoch 训练的逻辑。"""

@abstractmethod
def test(self):
    """测试的逻辑。"""

## 实现流程调度代码逻辑

调度器必须继承自 `Scheduler` 基础类，并且实现 `Scheduler` 中定义的接口，目前只需要实现 `_run` 一个接口。由于设计的是具体调度器实现的虚拟基础类，所以这里将其设置为一个 `ABCMeta` 类。

In [ ]:
from alphafed import logger
from alphafed.scheduler import Scheduler
from alphafed.data_channel import SharedFileDataChannel


class SimpleFedAvgScheduler(Scheduler, metaclass=ABCMeta):

    def __init__(self, rounds: int) -> None:
        super().__init__()
        # 自定义一些初始化参数，此处只定义了 rounds 一个参数，用于设置训练的轮数
        self.rounds = rounds

    def _run(self, id: str, task_id: str, is_initiator: bool = False, recover: bool = False):
        """运行调度器的入口。

        实际运行时由任务管理器负责传入接口参数，模拟环境下需要调试者自行传入模拟值。

        参数说明:
            id: 当前节点 ID
            task_id: 当前任务 ID
            is_initiator: 当前节点是否是任务发起方
            recover: 是否使用恢复模式启动
        """
        # 先记录传入的参数，由于本示例不支持恢复模式，可以忽略 recover
        self.id = id
        self.task_id = task_id
        self.is_initiator = is_initiator

        # 发起方作为聚合方，其它节点作为参与方
        if self.is_initiator:
            self._run_as_aggregator()
        else:
            self._run_as_collaborator()

    def _run_as_aggregator(self):
        """作为聚合方运行，具体实现后面介绍."""
        ...

    def _run_as_collaborator(self):
        """作为参与方运行，具体实现后面介绍."""
        ...

`_run_as_aggregator` 和 `_run_as_collaborator` 接口都是算法设计者需要提供实现的接口，并不是给算法调度器的使用者（实际训练模型的开发者）实现的，也不希望他们修改实现，所以都定义为私有的。而上一步整理的 `before_check_in` 等五个接口才是需要由使用者提供实现的，所以定义为公有的。后面的接口设计采用同样的原则。

然后把上一步整理定义的接口都加进去。

In [ ]:
from torch.nn import Module

class SimpleFedAvgScheduler(Scheduler, metaclass=ABCMeta):
    
    @abstractmethod
    def before_check_in(self):
        """完成集合前初始化本地资源。"""

    @abstractmethod
    def before_training(self):
        """训练开始前的初始化工作。"""

    @property
    @abstractmethod
    def model(self) -> Module:
        """获取训练使用的模型对象。"""

    @abstractmethod
    def train_an_epoch(self):
        """完成一个 epoch 训练的逻辑。"""

    @abstractmethod
    def test(self):
        """测试的逻辑。"""

以此为基础，可以开始逐步实现流程逻辑了。先考虑聚合方的情况，再贴一遍聚合方流程：
1. 上线，初始化本地资源；
2. 监听集合请求；
3. 收到集合请求后，记录参与者，并向请求方回复集合响应；
4. 统计参与者数量，如果集合完成发送开始训练的通知；
5. 完成训练初始化工作；
6. 计数训练轮次，发送全局模型；
7. 监听本地模型传输请求，收集本地模型，集齐全部本地模型后，聚合得到新的全局模型；
8. 检查训练轮次，判断训练是否完成；如果没有完成则跳到第 6 步，如果完成继续向下；
9. 使用测试数据做一次测试，得到全局模型的评估指标数据；
10. 上传模型文件和评估指标数据；
11. 通知任务管理器训练结束，完成训练退出。

In [ ]:
import io
import os
from abc import ABCMeta
from tempfile import TemporaryFile
from zipfile import ZipFile

import torch
from torch.utils.tensorboard import SummaryWriter

from alphafed.data_channel import SharedFileDataChannel
from alphafed.fs import get_result_dir
from alphafed.scheduler import Scheduler

from .contractor import CheckInEvent, SimpleFedAvgContractor


class SimpleFedAvgScheduler(Scheduler, metaclass=ABCMeta):

    def _run_as_aggregator(self):
        """作为聚合方运行."""
        # 初始化本地资源
        self.contractor = SimpleFedAvgContractor(task_id=self.task_id)
        self.data_channel = SharedFileDataChannel(contractor=self.contractor)

        self.collaborators = self.contractor.query_nodes()
        self.collaborators.remove(self.id)  # 把自己移出去
        self.checked_in = set()  # 记录集合的参与方
        self.result_dir = get_result_dir(self.task_id)
        self.log_dir = os.path.join(self.result_dir, 'tb_logs')  # 记录测试评估指标的目录
        self.tb_writer = SummaryWriter(log_dir=self.log_dir)  # 记录测试评估指标的 writter

        # 调用 before_check_in 执行用户自定义的额外初始化逻辑。
        # 聚合方与参与方的初始化逻辑可能会不一样，所以加一个 is_aggregator 参数已做区分。接口定义也据此更新。
        self.before_check_in(is_aggregator=True)
        self.push_log(f'节点 {self.id} 初始化完毕。')

        # 监听集合请求
        self.push_log('开始等待参与成员集合 ...')
        for _event in self.contractor.contract_events():
            if isinstance(_event, CheckInEvent):
                # 收到集合请求后，记录参与者，并向请求方回复集合响应
                self.checked_in.add(_event.peer_id)
                self.contractor.response_check_in(aggr_id=self.id, peer_id=_event.peer_id)
                self.push_log(f'成员 {_event.peer_id} 加入。')
                # 统计参与者数量，如果集合完成退出循环
                if len(self.collaborators) == len(self.checked_in):
                    break  # 退出监听循环
        self.push_log(f'参与成员集合完毕，共有 {len(self.checked_in)} 位参与者。')

        # 完成训练初始化工作
        self.model  # 初始化模型
        # 调用 before_training 执行用户自定义的额外初始化逻辑。
        # 聚合方与参与方的初始化逻辑可能会不一样，所以加一个 is_aggregator 参数已做区分。接口定义也据此更新。
        self.before_training(is_aggregator=True)
        self.push_log(f'节点 {self.id} 准备就绪，可以开始执行计算任务。')

        for _round in range(self.rounds):
            # 发送开始训练的通知
            self.contractor.start()
            self.push_log(f'第 {_round + 1} 轮训练开始。')
            # 计数训练轮次，发送全局模型
            with TemporaryFile() as f:
                torch.save(self.model.state_dict(), f)
                f.seek(0)
                self.push_log('开始发送全局模型 ...')
                self.data_channel.batch_send_stream(source=self.id,
                                                    target=self.collaborators,
                                                    data_stream=f.read(),
                                                    ensure_all_succ=True)
            self.push_log('发送全局模型完成。')
            # 监听本地模型传输请求，收集本地模型
            self.updates = []  # 记录本地模型参数更新
            self.push_log('开始等待收集本地模型 ...')
            training_results = self.data_channel.batch_receive_stream(
                receiver=self.id,
                source_list=self.collaborators,
                ensure_all_succ=True
            )
            for _source, _result in training_results.items():
                buffer = io.BytesIO(_result)
                state_dict = torch.load(buffer)
                self.updates.append(state_dict)
                self.push_log(f'收到来自 {_source} 的本地模型。')
            # 聚合得到新的全局模型
            self.push_log('开始执行参数聚合 ...')
            self._make_aggregation()
            self.push_log('参数聚合完成。')
            # 如果达到训练轮次，循环结束

        # 使用测试数据做一次测试，得到全局模型的评估指标数据
        # 测试时指定 TensorBoard 的 writter，否则用户使用自定义的 writter，无法控制日志文件目录。
        # 接口定义也据此更新。
        self.push_log('训练完成，测试训练效果 ...')
        self.run_test(writer=self.tb_writer)
        self.push_log('测试完成。')

        # 上传模型文件和评估指标数据
        # 打包记录测试时写入的所有 TensorBoard 日志文件
        self.push_log('整理计算结果，准备上传 ...')
        report_file = os.path.join(self.result_dir, "report.zip")
        with ZipFile(report_file, 'w') as report_zip:
            for path, _, filenames in os.walk(self.log_dir):
                rel_dir = os.path.relpath(path=path, start=self.result_dir)
                rel_dir = rel_dir.lstrip('.')  # ./file => file
                for _file in filenames:
                    rel_path = os.path.join(rel_dir, _file)
                    report_zip.write(os.path.join(path, _file), rel_path)
        report_file_path = os.path.abspath(report_file)
        # 记录训练后的模型参数
        model_file = os.path.join(self.result_dir, "model.pt")
        with open(model_file, 'wb') as f:
            torch.save(self.model.state_dict(), f)
        model_file_path = os.path.abspath(model_file)
        # 调用接口执行上传
        self.contractor.upload_metric_report(receivers=self.contractor.EVERYONE,
                                                report_file=report_file_path)
        self.contractor.upload_model(receivers=self.contractor.EVERYONE,
                                        model_file=model_file_path)
        self.push_log('计算结果上传完成。')

        # 通知任务管理器训练结束，完成训练退出
        self.contractor.notify_task_completion(result=True)
        self.contractor.close()
        self.push_log('计算任务完成。')

    def _make_aggregation(self):
        """执行参数聚合。"""
        # 模型参数清零
        global_params = self.model.state_dict()
        for _param in global_params.values():
            if isinstance(_param, torch.Tensor):
                _param.zero_()
        # 累加收到的本地模型参数
        for _update in self.updates:
            for _key in global_params.keys():
                global_params[_key].add_(_update[_key])
        # 求平均值获得新的全局参数
        count = len(self.collaborators)
        for _key in global_params.keys():
            if global_params[_key].dtype in (
                torch.uint8, torch.int8, torch.int16, torch.int32, torch.int64
            ):
                global_params[_key].div_(count, rounding_mode='trunc')
            else:
                global_params[_key].div_(count)
        self.model.load_state_dict(global_params)

接下来实现参与方的调度流程，再贴一遍参与方流程：
1. 上线，初始化本地资源；
2. 此时还不知道聚合方是谁，所以广播发送集合请求；
3. 监听集合响应，收到集合响应后，记录聚合方；
4. 完成训练初始化工作；
5. 监听开始训练消息或训练结束消息；如果收到开始训练消息，跳到第 6 步；如果收到训练结束消息，完成训练退出；
6. 监听传输全局模型消息；
7. 接收全局模型，使用其更新本地模型；
8. 在本地做一个 epoch 的训练，更新本地模型；
9. 将本地模型发送给聚合方；
16. 跳到第 5 步。

In [ ]:
from abc import ABCMeta
import io
from tempfile import TemporaryFile

import torch

from alphafed.data_channel import SharedFileDataChannel
from alphafed.scheduler import Scheduler

from .contractor import (CheckInResponseEvent, CloseEvent,
                         SimpleFedAvgContractor, StartEvent)


class SimpleFedAvgScheduler(Scheduler, metaclass=ABCMeta):

    def _run_as_collaborator(self):
        """作为参与方运行。"""
        # 初始化本地资源
        self.contractor = SimpleFedAvgContractor(task_id=self.task_id)
        self.data_channel = SharedFileDataChannel(contractor=self.contractor)

        # 调用 before_check_in 执行用户自定义的额外初始化逻辑。
        # 聚合方与参与方的初始化逻辑可能会不一样，所以加一个 is_aggregator 参数已做区分。接口定义也据此更新。
        self.before_check_in(is_aggregator=False)
        self.push_log(f'节点 {self.id} 初始化完毕。')

        # 广播发送集合请求
        self.push_log('发送集合请求，等待聚合方响应。')
        self.contractor.check_in(peer_id=self.id)
        # 监听集合响应，收到集合响应后，记录聚合方
        for _event in self.contractor.contract_events():
            if isinstance(_event, CheckInResponseEvent):
                self.aggregator = _event.aggr_id
                self.push_log('收到响应，集合成功。')
                break  # 退出监听循环

        # 完成训练初始化工作
        self.model  # 初始化模型
        # 调用 before_training 执行用户自定义的额外初始化逻辑。
        # 聚合方与参与方的初始化逻辑可能会不一样，所以加一个 is_aggregator 参数已做区分。接口定义也据此更新。
        self.before_training(is_aggregator=False)
        self.push_log(f'节点 {self.id} 准备就绪，可以开始执行计算任务。')

        while True:
            self.push_log('等待训练开始信号 ...')
            # 监听开始训练消息或训练结束消息；如果收到开始训练消息，跳到第 6 步；如果收到训练结束消息，完成训练退出
            for _event in self.contractor.contract_events():
                if isinstance(_event, StartEvent):
                    self.push_log('开始训练 ...')
                    break  # 退出监听循环
                elif isinstance(_event, CloseEvent):
                    self.push_log('训练完成。')
                    return  # 退出训练
            # 监听传输全局模型消息
            self.push_log('等待接收全局模型 ...')
            _, data_stream = self.data_channel.receive_stream(receiver=self.id,
                                                              source=self.aggregator)
            buffer = io.BytesIO(data_stream)
            new_state = torch.load(buffer)
            self.model.load_state_dict(new_state)
            self.push_log('接收全局模型成功。')
            # 在本地做一个 epoch 的训练，更新本地模型
            self.push_log('开始训练本地模型 ...')
            self.train_an_epoch()
            self.push_log('训练本地模型完成。')
            # 将本地模型发送给聚合方
            with TemporaryFile() as f:
                torch.save(self.model.state_dict(), f)
                f.seek(0)
                self.push_log('准备发送本地模型 ...')
                self.data_channel.send_stream(source=self.id,
                                              target=self.aggregator,
                                              data_stream=f.read())
                self.push_log('发送本地模型完成。')
            # 继续循环

至此，自定义的极简 FedAvg 算法调度器就实现了。将上面的代码整理好之后保存在一个 [scheduler.py](res/simple_fed_avg/scheduler.py) 文件中。之后会演示如何使用这个算法调度器训练模型。